In [1]:
import descarteslabs as dl
import gdal
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
import matplotlib.pyplot as plt
# %matplotlib inline

/Users/karlaking/miniconda2/envs/landcover/lib/python3.6/site-packages/osgeo/gdal.py:107: DeprecationWarning: gdal.py was placed in a namespace, it is now available as osgeo.gdal
  DeprecationWarning)


In [2]:
lake_taupo = {
  "type": "Feature",
  "properties": {},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          175.55877685546875,
          -39.27691581029594
        ],
        [
          176.319580078125,
          -39.27691581029594
        ],
        [
          176.319580078125,
          -38.638327308061875
        ],
        [
          175.55877685546875,
          -38.638327308061875
        ],
        [
          175.55877685546875,
          -39.27691581029594
        ]
      ]
    ]
  }
}

In [3]:
image_id = 'landsat:LC08:01:RT:TOAR:meta_LC08_L1TP_072087_20171211_20171212_01_RT_v1'

In [4]:
raster_arr, meta = dl.raster.ndarray(image_id, cutline=lake_taupo['geometry'],bands=['red','green','blue'], resolution=60)

In [6]:
rows, cols, band = raster_arr.shape

In [7]:
proj = meta['coordinateSystem']['wkt']

In [8]:
def create_mask_from_vector(vector_data_path, cols, rows, geo_transform,
                            projection, target_value=1):
    """Rasterize the given vector (wrapper for gdal.RasterizeLayer)."""
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')  # In memory dataset
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds


def vectors_to_raster(file_paths, rows, cols, geo_transform, projection):
    """Rasterize the vectors in the given directory in a single image."""
    labeled_pixels = np.zeros((rows, cols))
    print
    for i, path in enumerate(file_paths):
        label = i+1
        ds = create_mask_from_vector(path, cols, rows, geo_transform,
                                     projection, target_value=label)
        band = ds.GetRasterBand(1)
        labeled_pixels += band.ReadAsArray()
        ds = None
    return labeled_pixels


def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

In [9]:
scenes, ctx = dl.scenes.search(lake_taupo['geometry'],
                    products=["landsat:LC08:01:RT:TOAR"],
                    start_datetime="2017-12-11",
                    end_datetime="2017-12-12",
                    cloud_fraction=0.7
                   )

In [10]:
scenes

SceneCollection of 1 scene
  * Dates: Dec 11, 2017 to Dec 11, 2017
  * Products: landsat:LC08:01:RT:TOAR: 1

In [12]:
ctx

AOI(geometry=<shapely.geom... 0x1a1711c208>,
    resolution=15,
    crs='EPSG:32660',
    align_pixels=True,
    bounds=(175.55877685546875, -39.27691581029594, 176.319580078125, -38.638327308061875),
    shape=None)

In [13]:
lowres_context = ctx.assign(resolution=60, crs="EPSG:32760")

In [14]:
lowres_context

AOI(geometry=<shapely.geom... 0x1a17121dd8>,
    resolution=60,
    crs='EPSG:32760',
    align_pixels=True,
    bounds=(175.55877685546875, -39.27691581029594, 176.319580078125, -38.638327308061875),
    shape=None)

In [15]:
proj = 'PROJCS["WGS 84 / UTM zone 60N", GEOGCS["WGS 84", DATUM["WGS_1984", SPHEROID["WGS 84",6378137,298.257223563, AUTHORITY["EPSG","7030"]], AUTHORITY["EPSG","6326"]], PRIMEM["Greenwich",0, AUTHORITY["EPSG","8901"]], UNIT["degree",0.0174532925199433, AUTHORITY["EPSG","9122"]], AUTHORITY["EPSG","4326"]], PROJECTION["Transverse_Mercator"], PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",177],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0], UNIT["metre",1, AUTHORITY["EPSG","9001"]], AXIS["Easting",EAST], AXIS["Northing",NORTH], AUTHORITY["EPSG","32660"]]'

In [ ]:
stack = scenes.stack("red green blue", lowres_context)

In [ ]:
stack

In [ ]:
type(scenes[0])

In [12]:
arr = scenes[0].ndarray("red green blue",lowres_context)

In [14]:
# bands, rows, cols = arr.shape

In [ ]:
# n_bands, rows, cols = stack[0].shape

In [15]:
geo_transform = scenes[0].properties.geotrans

In [34]:

print(geo_transform)

[330592.5, 15.0, 0.0, -4189492.5, 0.0, -15.0]


In [ ]:
# import numpy as np
composite = np.ma.median(stack, axis=0)
dl.scenes.display(composite, title="Lake Taupo Composite")

Load data

In [16]:
train_data_path = "data/train/"

In [24]:
files = [f for f in os.listdir(train_data_path) if f.endswith('.shp')]
classes = [f.split('.')[0] for f in files]
print("The class are {} and {}.".format(classes[0],classes[1]))
shapefiles = [os.path.join(train_data_path, f)
              for f in files if f.endswith('.shp')]

labeled_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform,
                                   proj)
is_train = np.nonzero(labeled_pixels)
training_labels = labeled_pixels[is_train]
training_samples = raster_arr[is_train]

The class are urban and water.


In [27]:
training_samples

numpy.ndarray

In [30]:
proj

'PROJCS["UTM Zone 60, Northern Hemisphere",\n    GEOGCS["WGS 84",\n        DATUM["WGS_1984",\n            SPHEROID["WGS 84",6378137,298.257223563,\n                AUTHORITY["EPSG","7030"]],\n            AUTHORITY["EPSG","6326"]],\n        PRIMEM["Greenwich",0,\n            AUTHORITY["EPSG","8901"]],\n        UNIT["degree",0.0174532925199433,\n            AUTHORITY["EPSG","9122"]],\n        AUTHORITY["EPSG","4326"]],\n    PROJECTION["Transverse_Mercator"],\n    PARAMETER["latitude_of_origin",0],\n    PARAMETER["central_meridian",177],\n    PARAMETER["scale_factor",0.9996],\n    PARAMETER["false_easting",500000],\n    PARAMETER["false_northing",0],\n    UNIT["Meter",1]]'

In [31]:
trained_classifier = RandomForestClassifier(n_jobs=-1)
trained_classifier.fit(training_samples, training_labels)

ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required.